## Imports and setup

In [ ]:
!pip install wandb pytorch_lightning torchmetrics polars catboost

In [ ]:
import os
import random
import sys
import collections
import importlib
from abc import ABC, abstractmethod

import wandb
import numpy as np
import pandas as pd
import polars as pls
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tqdm import tqdm, trange

import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error


import catboost
from catboost import Pool, CatBoostRegressor
from catboost import EShapCalcType, EFeaturesSelectionAlgorithm


import torch
import torchmetrics
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

from torchmetrics import R2Score, MeanSquaredError, MeanAbsolutePercentageError

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

from collections import deque, defaultdict

In [ ]:
# optional
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
class ExponentionalAverager:
    def __init__(self, start_value, window_size):
        assert window_size >= 0
        self.val = start_value
        self.alpha = 2. / (window_size + 1)
        self.window_size = window_size

    def update(self, val):
        self.val = self.alpha * val + (1. - self.alpha) * self.val

    def get(self):
        return self.val

## Datasets

In [ ]:
!rm -rf qber-forecasting
!rm -rf deep_qber
!git clone https://github.com/rmnigm/qber-forecasting.git
!cp -r qber-forecasting/deep_qber deep_qber

Cloning into 'qber-forecasting'...
remote: Enumerating objects: 245, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 245 (delta 35), reused 32 (delta 12), pack-reused 176
Receiving objects: 100% (245/245), 46.51 MiB | 10.27 MiB/s, done.
Resolving deltas: 100% (114/114), done.


In [ ]:
path = "/content/qber-forecasting/datasets/qber_with_outliers.csv"
raw_dataframe = pd.read_csv(path)

info_path = "/content/qber-forecasting/datasets/outliers_info.csv"
info_dataframe = pd.read_csv(info_path)

In [ ]:
with_steps = (
    raw_dataframe
    .set_index('index')
    .join(info_dataframe.set_index('index')[['steps_to_anomaly']],
          on='index',
          how='left',
          rsuffix='_info'
          )
    )

with_steps['outliers'] = (with_steps['steps_to_anomaly'] == 0).astype(int)
wo_junk = with_steps.drop(columns='steps_to_anomaly')

In [ ]:
from deep_qber.dataset import BaseDataset

In [ ]:
class TorchDataset(torch.utils.data.Dataset):
    def __init__(self, data, device='cpu'):
        self.indices, self.X, self.Y = [], [], []
        for point in data:
            i, data = point
            x_lag, x_latest, y = data
            self.indices.append(i)
            self.X.append((x_lag, x_latest))
            self.Y.append(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return tuple(x.to(device) for x in self.X[idx]), self.Y[idx].to(device)

In [ ]:
class TorchDatasetInterface(BaseDataset):
    def __init__(self,
                 dataframe,
                 target_column,
                 anomaly_column,
                 shuffle=True,
                 train_size=0.75,
                 batch_size=64,
                 window_size=10,
                 dtype=torch.float32,
                 ):
        super().__init__(dataframe=dataframe,
                         target_column=target_column,
                         anomaly_column=anomaly_column,
                         window_size=window_size,
                         train_size=train_size)
        self.dtype = dtype
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.train = None
        self.test = None

    def transform(self, subset):
        lag, latest = subset.iloc[:-1], subset.iloc[-1]
        y = torch.tensor(
            [latest[self.target_column]],
            dtype=self.dtype
        )
        x_latest = torch.tensor(
            latest.drop(self.target_column).values,
            dtype=self.dtype
        )
        x_lag = torch.tensor(
            lag.values,
            dtype=self.dtype
        )
        return x_lag, x_latest, y

    def assemble(self, dataset):
        self.dataset = dataset
        torch.save(self.dataset, 'qber_dataset.pt')

    def load(self):
        self.dataset = torch.load('qber_dataset.pt')

    def get_dataloaders(self, device='cpu', anomaly_split=True):
        loaders = []
        normal, anomaly = [], []
        for point in self.dataset:
            i, data, anomaly_label = point
            if anomaly_split and anomaly_label:
                anomaly.append((i, data))
            else:
                normal.append((i, data))
        subsets = [normal]
        if anomaly:
            subsets.append(anomaly)
        for subset in subsets:
            train, test = self.split(subset, train_size=self.train_size)
            train_dataset = TorchDataset(train, device=device)
            train_loader = torch.utils.data.DataLoader(train_dataset,
                                                       shuffle=self.shuffle,
                                                       batch_size=self.batch_size)

            test_dataset = TorchDataset(test, device=device)
            test_loader = torch.utils.data.DataLoader(test_dataset,
                                                      shuffle=self.shuffle,
                                                      batch_size=self.batch_size)
            loaders.append((train_loader, test_loader))

        return loaders

In [ ]:
window_size = 20
start = 20
end = len(raw_dataframe)
target_column = 'e_mu_current'
anomaly_column = 'outliers'

In [ ]:
# from deep_qber.dataset import TorchDatasetInterface, ClassicModelDataset

In [ ]:
datavault = TorchDatasetInterface(wo_junk,
                                  target_column,
                                  anomaly_column='outliers',
                                  window_size=20,
                                  train_size=0.75
                                  )

In [ ]:
datavault.setup(20, end, from_files=False)
dataloaders = datavault.get_dataloaders(anomaly_split=True, device=device)

100%|██████████| 184830/184830 [03:56<00:00, 781.92it/s]


In [ ]:
config = {
    "learning_rate": 1e-5,
    "look_back": window_size,
    "hidden_size": 256,
    "output_size": 1,
    "input_size": 7,
    "batch_size": 256,
    "epochs": 10,
    "loss": "MSE",
    "scaler": None,
    "train_size": 0.75,
}

loss = nn.MSELoss()
scaler = None

## Models (Torch)

In [ ]:
class ModelInterfaceTS(nn.Module):
    def __init__(self, model, device):
        super().__init__()
        self.model = model
        self.metrics = {
            "MSE": torchmetrics.MeanSquaredError().to(device),
            "R2Score": torchmetrics.R2Score().to(device),
            "MAPE": torchmetrics.MeanAbsolutePercentageError().to(device),
        }

    def forward(self, x):
        return self.model(x)

    def get_metrics(self, predictions, labels):
        preds_f, labels_f = predictions.float(), labels.float()
        return {k: v(preds_f, labels_f) for k, v in self.metrics.items()}

In [ ]:
class ModuleTS(pl.LightningModule):
    def __init__(self, model, loss, lr=1e-5):
        super().__init__()
        self.model = model
        self.loss = loss
        self.lr = lr
        self.loss_multiplier = 1e4
        self.save_hyperparameters(ignore=['model'])

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        data, target = train_batch
        predictions = self.forward(data)
        loss = self.loss_multiplier * self.loss(predictions, target)
        self.log("Train Loss", loss, prog_bar=True)
        metrics = self.model.get_metrics(predictions, target)
        for k, v in metrics.items():
            self.log(f"Train {k}", v, prog_bar=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        data, target = val_batch
        preds = self.forward(data)
        loss = self.loss_multiplier * self.loss(preds, target)
        metrics = self.model.get_metrics(preds, target)
        self.log("Validation Loss", loss, prog_bar=True)
        for k, v in metrics.items():
            self.log(f"Validation {k}", v, prog_bar=True)

In [ ]:
(train_normal_loader, test_normal_loader), (train_anomaly_loader, test_anomaly_loader) = dataloaders

In [ ]:
from deep_qber.model import Extractor, ExtractorExod, ExtractorLSTM

In [ ]:
# for model_type in (Extractor, ExtractorExod, ExtractorLSTM):
#     model = model_type(
#         look_back=config["look_back"],
#         output_size=config["output_size"],
#         input_size=config["input_size"],
#         hidden_size=config["hidden_size"],
#         ).to(device)
#     batch = next(iter(train_normal_loader))
#     x, c = batch
#     print(x[0].shape, x[1].shape)
#     print(model(x).shape, c.shape)
#     assert c.shape == model(x).shape

In [ ]:
model = ExtractorLSTM(
    look_back=config["look_back"],
    output_size=config["output_size"],
    input_size=config["input_size"],
    hidden_size=config["hidden_size"],
    )

In [ ]:
def run_experiment(train_loader, test_loader, model, loss, config, device):
    with wandb.init(project="qber_v2",
                    entity="rmnigm",
                    settings=wandb.Settings(start_method="thread"),
                    config=config,
                    ) as run:
        wandb_logger = WandbLogger(log_model='all')
        checkpoint_callback = ModelCheckpoint(monitor="Validation R2Score", mode="max")

        epochs = config["epochs"]

        model_interface = ModelInterfaceTS(model, device)
        module = ModuleTS(model_interface, loss, lr=config["learning_rate"])

        trainer = pl.Trainer(logger=wandb_logger,
                            callbacks=[checkpoint_callback],
                            accelerator="gpu",
                            max_epochs=epochs,
                            )

        trainer.fit(module, train_loader, test_loader)

        run.finish()

In [ ]:
config["model"] = "lstm; normal data"
run_experiment(train_normal_loader, test_normal_loader, model, loss, config, device)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:398: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/parsing.py:197: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INF

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:490: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


Train Loss,▂▃▃▃▂▃▅▃▄▂▅▂▃█▁▁█▂▄▄▃▁▃▅▄▃▁▅▂▃▃▃▄▁▃▃▃▃▂▂
Train MAPE,▁▂▂▂▂▂▂▁▂▂█▁▂█▁▁▂▁▂▂▂▁▁▂▂▁▁▁▂▂▁▁▂▁▁▁▁▁▁▁
Train MSE,▂▃▃▃▂▃▅▃▄▂▅▂▃█▁▁█▂▄▄▃▁▃▅▄▃▁▅▂▃▃▃▄▁▃▃▃▃▂▂
Train R2Score,▆▆▆▆▇▇▇▇▄▆▆▇▇▇▇▇▁▇▆▇▇▇▃▇██▇█▆▆▇▇▇▇▇▇▇█▆▇
Validation Loss,▆▄▄▇▃▁▁▅█▄
Validation MAPE,▆▅▅█▄▂▁▆▇▅
Validation MSE,▆▄▄▇▃▁▁▅█▄
Validation R2Score,▃▄▄▁▅▇█▃▂▄
epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Train Loss,0.02843


In [ ]:
config["model"] = "lstm; anomaly data"
run_experiment(train_anomaly_loader, test_anomaly_loader, model, loss, config, device)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ModelInterfaceTS | 339 K 
1 | loss  | MSELoss          | 0     
-------------------------------------------
339 K     Trainable params
0         Non-trainable params
339 K     Total params
1.358     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:281: PossibleUserWarning: The number of training batches (35) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


Train Loss,▂▄█▂▁▂▁
Train MAPE,▄▄▂▂▁██
Train MSE,▂▄█▂▁▂▁
Train R2Score,▁▄▃█▆▇█
Validation Loss,██▅▇▄▄▆▂▃▁
Validation MAPE,▅▄█▃█▆▁▇▃▅
Validation MSE,██▅▇▄▄▆▂▃▁
Validation R2Score,▁▁▃▂▅▆▄▅▆█
epoch,▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
trainer/global_step,▁▁▂▂▃▃▄▄▅▅▆▆▆▇▇██
Train Loss,1.56934


## GBRT Dataset

In [ ]:
datavault = ClassicModelDataset(wo_junk,
                                target_column,
                                anomaly_column='outliers',
                                window_size=20,
                                train_size=0.75
                                )

In [ ]:
datavault.setup(20, len(wo_junk), from_files=False)
pools, targets = datavault.get_catboost_pools(anomaly_split=False)

In [ ]:
def evaluate_metrics(y_true, y_pred, model_name='Unnamed model'):
    metrics = {
        "MAPE": mean_absolute_percentage_error(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "R2 Score": r2_score(y_true, y_pred)
    }
    print(f'MAPE value:')
    print(f'{metrics["MAPE"]:.8f}')
    print(f'MSE value:')
    print(f'{metrics["MSE"]:.8f}')
    print(f'RMSE value:')
    print(f'{metrics["RMSE"]:.8f}')
    print(f'R2 Score value:')
    print(f'{metrics["R2 Score"]:.8f}')
    metrics['model'] = model_name
    return metrics

In [ ]:
normal_train_pool, normal_test_pool = pools[0]
normal_train_y, normal_test_y = targets[0]

anomaly_train_pool, anomaly_test_pool = pools[1]
anomaly_train_y, anomaly_test_y = targets[1]

## Models (GBRT)

#### Normal

In [ ]:
normal_regressor = CatBoostRegressor()

In [ ]:
features = list(filter(lambda col: col != target_column, datavault.schema))

In [ ]:
normal_regressor.fit(normal_train_pool, eval_set=normal_test_pool)

In [ ]:
summary = normal_regressor.select_features(
    normal_train_pool,
    eval_set=normal_test_pool,
    features_for_select=f'0-{len(features)-1}',
    num_features_to_select=40,
    steps=10,
    algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
    shap_calc_type=EShapCalcType.Regular,
    train_final_model=True,
    plot=True
)

In [ ]:
normal_preds = normal_regressor.predict(normal_test_pool)

In [ ]:
normal_metrics = evaluate_metrics(normal_test_y, normal_preds)

MAPE value:
0.09732960
MSE value:
0.00000105
RMSE value:
0.00102468
R2 Score value:
0.52593793


In [ ]:
# ema = ExponentionalAverager(0.01, 5)
# ema_vals = []
# for t in test_y:
#     ema_vals.append(ema.get())
#     ema.update(t)

# ema_metrics = evaluate_metrics(test_y, ema_vals)

In [ ]:
feature_imp = pd.DataFrame({
    'feature_importance': normal_regressor.get_feature_importance(train_pool),
    'feature_names': features
})
feature_imp = feature_imp.sort_values(by=['feature_importance'], ascending=False)
feature_imp.head(20)

#### Anomaly

In [ ]:
anomaly_regressor = CatBoostRegressor()

In [ ]:
summary = anomaly_regressor.select_features(
    anomaly_train_pool,
    eval_set=anomaly_test_pool,
    features_for_select=f'0-{len(features)-1}',
    num_features_to_select=40,
    steps=10,
    algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
    shap_calc_type=EShapCalcType.Regular,
    train_final_model=True,
    plot=True
)

In [ ]:
anomaly_preds = anomaly_regressor.predict(anomaly_test_pool)
anomaly_metrics = evaluate_metrics(anomaly_test_y, anomaly_preds)

MAPE value:
0.15204956
MSE value:
0.00093888
RMSE value:
0.03064108
R2 Score value:
0.74559747


In [ ]:
feature_imp = pd.DataFrame({
    'feature_importance': anomaly_regressor.get_feature_importance(anomaly_train_pool),
    'feature_names': features
})
feature_imp = feature_imp.sort_values(by=['feature_importance'], ascending=False)

feature_imp.head(30)

#### Total

In [ ]:
preds = list(anomaly_preds) + list(normal_preds)
true_labels = list(anomaly_test_y) + list(normal_test_y)

total_metrics = evaluate_metrics(true_labels, preds)

MAPE value:
0.09820117
MSE value:
0.00001599
RMSE value:
0.00399845
R2 Score value:
0.76757622
